# Study Design: Customer Segmentation

본 노트북은 고객 세분화 연구를 위한 **2-Track Study Design**을 정의한다.

## 목차

1. [개요 및 Framework](#1.-Overview-&-Framework)
2. [Track 관계 및 실무 활용](#2.-Track-Relationships-&-Practical-Usage)
3. [Track 1: Customer Understanding](#3.-Track-1:-Customer-Understanding)
   - 3.1 Methods
   - 3.2 Operational Definitions
   - 3.3 Features
   - 3.4 Evaluation
4. [Track 2: Causal Targeting](#4.-Track-2:-Causal-Targeting)
   - 4.1 Methods
   - 4.2 Operational Definitions (2 Scenarios)
   - 4.3 Causal Graph
   - 4.4 Features
   - 4.5 Evaluation
5. [기간 및 검증 설계](#5.-Period-&-Validation-Design)
6. [구현 계획](#6.-Implementation-Plan)
7. [Cohort 정의 및 비교](#7.-Cohort-Definition-&-Comparison)

---

## 1. 개요 및 Framework

### 1.1 Framework 구조

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         2-TRACK STUDY DESIGN                                │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  TRACK 1: CUSTOMER UNDERSTANDING (Descriptive)                              │
│  ═══════════════════════════════════════════════                            │
│                                                                             │
│  Step 1.1: Customer Profiling                                               │
│    - Factor Analysis (NMF) → Latent Dimension 발견                          │
│    - Clustering → Base Segment 도출                                         │
│                                                                             │
│  Step 1.2: Value × Need Integration                                         │
│    - Value Layer: CLV, Engagement                                           │
│    - Need Layer: Behavior, Category Preference                              │
│    - Integration: Value-Need Matrix 또는 2D Segmentation                    │
│                                                                             │
│  Output: "누구에게(Value) 무엇을(Need) 제공할 것인가?"                        │
│                                                                             │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  TRACK 2: CAUSAL TARGETING                                                  │
│  ═════════════════════════                                                  │
│                                                                             │
│  Step 2.1: HTE Analysis                                                     │
│    - Campaign Effect Heterogeneity                                          │
│    - Covariates: Track 1 Segment + Raw Feature                              │
│                                                                             │
│  Step 2.2: Optimal Policy                                                   │
│    - Targeting Rules                                                        │
│    - Policy Value Estimation                                                │
│                                                                             │
│  Output: "타겟팅을 어떻게 최적화할 것인가?"                                   │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 1.2 설계 근거

2-Track 접근법은 다음을 분리한다:
- **기술적 분석** (고객 이해)과
- **인과 분석** (개입 최적화)

이 분리는 다음을 가능하게 한다:
1. 전문성에 따라 다른 팀이 적절한 Track에서 작업 가능
2. Track 1이 즉각적인 마케팅 가치 제공
3. Track 2가 Track 1 세그먼트를 기반으로 인과적 인사이트 구축

---

## 2. Track 관계 및 실무 활용

### 2.1 Track 관계

```
Step 1.1 (Customer Profiling) ──────────────────────────┐
    │                                            │
    │ (유용하면 사용, 그렇지 않으면 독립 분석)       │ (선택적 Factor 사용)
    ↓                                            ↓
Step 1.2 (Value × Need)                   Track 2: Steps 2.1-2.2
    │                                            ↑
    │ (순차: Track 1 먼저)                        │
    └─────── Segment를 Moderator로 활용 ─────────┘
```

**주요 의존성:**
- **Step 1.1 → Step 1.2**: NMF Factor가 유용하면 사용, 그렇지 않으면 독립 분석
- **Track 1 → Track 2**: 순차 실행 (Track 2 전에 Track 1 완료)
- **Cross-Track**: Track 1 Segment가 Track 2에서 HTE Moderator 역할

### 2.2 실무 활용 비교

| 측면 | Track 1 (Descriptive) | Track 2 (Causal) |
|------|----------------------|------------------|
| **핵심 질문** | "이 고객은 누구인가?" | "이 개입이 그들에게 효과가 있을까?" |
| **주요 사용자** | 마케팅, CRM, 전략 | Data Science, 최적화 |
| **사용 시점** | 캠페인 기획, 전략 | 캠페인 실행, A/B 최적화 |
| **난이도** | 낮음 | 높음 |
| **설명 가능성** | "Premium Fresh Lover 세그먼트" | "이 고객의 CATE = 0.15" |
| **사용 빈도** | 높음 (일상 운영) | 중간 (특정 시나리오) |
| **조직 요구사항** | 기본 분석 역량 | Causal Thinking 문화 필요 |

### 2.3 마케터 관점

**Track 1 (거의 확실히 활용):**
- 직관적: "High-Value + Deal Seeker" → 즉시 이해 가능
- 실행 용이: 세그먼트 선택 → 오퍼 설계
- 커뮤니케이션 용이: 경영진 및 타 부서에 설명 가능
- 안정적: 세그먼트가 자주 변하지 않음

**Track 2 (조직 성숙도에 따라 다름):**
- 개념적 이해 필요: "Treatment Effect", "Uplift"
- 실험 인프라 필요: A/B Testing 시스템
- 해석 갭: 숫자를 실행 가능한 인사이트로 변환 필요
- 높은 가치: 적절히 구현 시 ROI 최적화 가능

---

## 3. Track 1: Customer Understanding

### 3.1 Methods

#### Step 1.1: Customer Profiling Analysis

**목표:** 잠재 고객 차원을 발견하고 Base Segment를 도출한다.

**방법:**
1. **Non-negative Matrix Factorization (NMF)**
   - Input: 33개 전체 고객 Feature
   - Output: Latent Factor Score 및 Loading
   - 해석: Loading 패턴을 기반으로 Factor 명명

2. **Clustering**
   - Algorithm: K-Means 또는 GMM
   - Input: NMF Factor Score (또는 Raw Feature)
   - Output: Base 고객 세그먼트

**연구 질문:**
- 고객 행동을 설명하는 Latent Dimension은 무엇인가?
- 데이터에 어떤 고객 유형이 존재하는가?

#### Step 1.2: Value × Need Integration (Optional)

**목표:** 고객 가치와 니즈를 결합한 실행 가능한 세그먼트 생성.

**Step 1.1 이후 이 단계가 선택적인 이유:**

Step 1.1 (Customer Profiling)은 이미 NMF를 통해 Value와 Need 차원을 모두 포착한다:
- **Value Factor**: F2 (Loyal Regular), F3 (Big Basket)가 고객 가치 포착
- **Need Factor**: F4 (Fresh Focused), F5 (Health & Beauty)가 선호 포착
- **Bubble Chart**: 모든 Factor 쌍의 2D 비교 가능

**Single-Space NMF의 한계:**
```
현재 (1.1):                    대안 (1.2):
Customer → [All Features]      Customer → [Value Features] → Value Tier
         → NMF                          → [Need Features]  → Need Type
         → Segment                      → (Value, Need) Matrix
```

Seg1 (VIP) 내에서 다음을 쉽게 구분할 수 없음:
- VIP + Fresh 선호
- VIP + H&B 선호

"동일 가치 계층, 다른 니즈"에 대한 Cross-sell 타겟팅이 제한됨.

**Value × Need가 제공하는 것:**
1. **명시적 2축 분리**: "누구에게(Value)" × "무엇을(Need)"에 대한 직접적 답변
2. **Cross-sell 타겟팅**: 동일 Value Tier 내 니즈 확장 기회 식별
3. **계층적 전략**: Value 기반 우선순위 설정 + Need 기반 오퍼 설계

**Value Layer (RFM + Engagement):**
- Core Feature: recency, frequency, monetary_sales
- Extended: monetary_avg_basket_sales, purchase_regularity, week_coverage
- CLV 추정: Historical, RFM 기반, 또는 Predictive (BG/NBD, Gamma-Gamma)

**Need Layer (Behavioral + Category):**
- Behavioral: discount_rate, discount_usage_pct, private_label_ratio, n_departments
- Category: share_grocery, share_fresh, share_bakery, share_health_beauty, share_alcohol

**Integration 옵션:**
- Option A: Value-Need Matrix (Value Tier × Need Cluster)
- Option B: Sequential Segmentation (Value Tier → Tier 내 Need Cluster)

**권고:** Track 2 (Causal Targeting)로 진행. Cross-sell 최적화가 우선순위가 되면 Value × Need 재검토.

### 3.2 Operational Definitions

| 항목 | 정의 |
|------|------|
| **Analysis Unit** | `household_key` (고객) |
| **Cohort** | Week 1-102에 최소 1회 구매 고객 (~2,500명) |
| **Observation Period** | Week 1-102 (전체 102주) |
| **Features** | 33개 Base Feature (RFM, Behavioral, Category, Time) |
| **Output** | 고객 세그먼트, Factor Score |

### 3.3 Features

#### Base Features (33개)

| 그룹 | 개수 | Features |
|------|------|----------|
| **Recency** | 6 | recency, recency_weeks, active_last_4w, active_last_12w, days_between_purchases_avg, days_between_purchases_std |
| **Frequency** | 6 | frequency, frequency_per_week, frequency_per_month, transaction_count, weeks_with_purchase, purchase_regularity |
| **Monetary** | 7 | monetary_sales, monetary_actual, monetary_avg_basket_sales, monetary_avg_basket_actual, monetary_std, monetary_per_week, coupon_savings_ratio |
| **Behavioral** | 7 | discount_rate, discount_usage_pct, private_label_ratio, n_departments, n_products, avg_items_per_basket, avg_products_per_basket |
| **Category** | 6 | share_grocery, share_fresh, share_bakery, share_health_beauty, share_alcohol, share_other |
| **Time** | 1 | week_coverage |

#### Feature 할당

| Step | Feature 범위 | 개수 |
|------|-------------|------|
| **Step 1.1 Customer Profiling** | 33개 전체 Feature → NMF | 33 |
| **Step 1.2a Value** | RFM + Engagement | ~21 |
| **Step 1.2b Need** | Behavioral + Category | 13 |

### 3.3.1 Feature 정의 (상세)

#### Track 1: Base Features (33개)

| Category | Subcategory | 변수 | 설명 | 수식 |
|----------|-------------|------|------|------|
| **RFM** | Recency | `recency` | 마지막 구매 후 경과 일수 | max(DAY) - last_purchase_day |
| RFM | Recency | `recency_weeks` | 마지막 구매 후 경과 주수 | max(WEEK) - last_purchase_week |
| RFM | Recency | `active_last_4w` | 최근 4주 활성 (binary) | 1 if last_week >= max_week - 4 else 0 |
| RFM | Recency | `active_last_12w` | 최근 12주 활성 (binary) | 1 if last_week >= max_week - 12 else 0 |
| RFM | Recency | `days_between_purchases_avg` | 구매 간 평균 일수 | mean(diff(purchase_days)) |
| RFM | Recency | `days_between_purchases_std` | 구매 간 일수 표준편차 | std(diff(purchase_days)) |
| **RFM** | Frequency | `frequency` | 총 방문 횟수 (Basket) | nunique(BASKET_ID) |
| RFM | Frequency | `frequency_per_week` | 주당 평균 방문 횟수 | frequency / tenure_weeks |
| RFM | Frequency | `frequency_per_month` | 월당 평균 방문 횟수 | frequency_per_week × 4.33 |
| RFM | Frequency | `transaction_count` | 총 거래 건수 | count(transactions) |
| RFM | Frequency | `weeks_with_purchase` | 구매가 있는 주 수 | nunique(WEEK_NO) |
| RFM | Frequency | `purchase_regularity` | 구매 규칙성 (0-1) | weeks_with_purchase / tenure_weeks |
| **RFM** | Monetary | `monetary_sales` | 총 매출 (할인 후) | sum(SALES_VALUE) |
| RFM | Monetary | `monetary_actual` | 총 실제 결제액 | sum(ACTUAL_SPENT) |
| RFM | Monetary | `monetary_avg_basket_sales` | 평균 장바구니 매출 | mean(basket_sales) |
| RFM | Monetary | `monetary_avg_basket_actual` | 평균 장바구니 결제액 | mean(basket_actual) |
| RFM | Monetary | `monetary_std` | 장바구니 매출 표준편차 | std(basket_sales) |
| RFM | Monetary | `monetary_per_week` | 주당 평균 매출 | monetary_sales / tenure_weeks |
| RFM | Monetary | `coupon_savings_ratio` | 쿠폰 절약 비율 | total_coupon_disc / monetary_sales |
| **Behavioral** | Price Sensitivity | `discount_rate` | 전체 할인율 | total_discount / (sales + total_discount) |
| Behavioral | Price Sensitivity | `discount_usage_pct` | 할인 거래 비율 | discount_transactions / total_transactions |
| Behavioral | Brand | `private_label_ratio` | PB 구매 비율 | PL_count / (PL_count + NB_count) |
| Behavioral | Basket | `n_departments` | 방문 부서 수 | nunique(DEPARTMENT) |
| Behavioral | Basket | `n_products` | 구매 고유 상품 수 | nunique(PRODUCT_ID) |
| Behavioral | Basket | `avg_items_per_basket` | 장바구니당 평균 아이템 수 | total_quantity / n_baskets |
| Behavioral | Basket | `avg_products_per_basket` | 장바구니당 평균 고유 상품 수 | n_products / n_baskets |
| **Category** | Grocery | `share_grocery` | Grocery 카테고리 매출 비중 | grocery_sales / total_sales |
| Category | Fresh | `share_fresh` | Fresh 카테고리 매출 비중 | fresh_sales / total_sales |
| Category | Bakery | `share_bakery` | Bakery 카테고리 매출 비중 | bakery_sales / total_sales |
| Category | Health&Beauty | `share_health_beauty` | Health & Beauty 카테고리 매출 비중 | hb_sales / total_sales |
| Category | Alcohol | `share_alcohol` | Alcohol 카테고리 매출 비중 | alcohol_sales / total_sales |
| Category | Other | `share_other` | 기타 카테고리 매출 비중 | other_sales / total_sales |
| **Time** | Coverage | `week_coverage` | 구매 주 커버리지 비율 | n_weeks_active / (week_range + 1) |

**Macro Category Mapping:**
- `grocery`: GROCERY, FROZEN GROCERY, GRO BAKERY
- `fresh`: PRODUCE, MEAT, MEAT-PCKGD, SEAFOOD, DELI, DAIRY DELI, SALAD BAR, ...
- `bakery`: PASTRY
- `health_beauty`: DRUG GM, NUTRITION, COSMETICS, RX, HBC, ...
- `alcohol`: SPIRITS

---

#### Track 1: Reduced Features (19개)

Step 1.1의 NMF에 사용되는 Feature (중복 제거를 위한 축소 셋):

| Category | 변수 | 선택 근거 |
|----------|------|----------|
| Recency (2) | `recency`, `days_between_purchases_avg` | Core 메트릭, 중복 변형 제거 |
| Frequency (3) | `frequency`, `frequency_per_week`, `purchase_regularity` | 절대, 상대, 규칙성 측정 |
| Monetary (4) | `monetary_sales`, `monetary_avg_basket_sales`, `monetary_std`, `coupon_savings_ratio` | 총액, 평균, 변동성, 절약 |
| Behavioral (5) | `discount_usage_pct`, `private_label_ratio`, `n_departments`, `n_products`, `avg_items_per_basket` | 가격 민감도, 브랜드 선호, 다양성, 장바구니 크기 |
| Category (5) | `share_grocery`, `share_fresh`, `share_bakery`, `share_health_beauty`, `share_alcohol` | share_other 제외 (합이 1) |

---

#### Track 2: Additional Features

| Category | Subcategory | 변수 | 설명 | 수식 |
|----------|-------------|------|------|------|
| **Exposure** | Display | `display_exposure_rate` | Display 노출 상품 구매 비율 | display_exposed_count / n_transactions |
| Exposure | Display | `display_intensity_avg` | 노출 시 평균 Display 강도 (1-10) | mean(display_level) when exposed |
| Exposure | Mailer | `mailer_exposure_rate` | Mailer 노출 상품 구매 비율 | mailer_exposed_count / n_transactions |
| **Campaign** | Targeting | `targeted_typeA` | TypeA 캠페인 타겟팅 여부 (binary) | 1 if targeted by TypeA else 0 |
| Campaign | Targeting | `targeted_typeB` | TypeB 캠페인 타겟팅 여부 (binary) | 1 if targeted by TypeB else 0 |
| Campaign | Targeting | `targeted_typeC` | TypeC 캠페인 타겟팅 여부 (binary) | 1 if targeted by TypeC else 0 |
| Campaign | Count | `n_campaigns_targeted` | 타겟팅된 총 캠페인 수 | sum(TypeA + TypeB + TypeC) |
| Campaign | Count | `n_typeA_campaigns` | 타겟팅된 TypeA 캠페인 수 | count(TypeA campaigns) |
| **Outcome** | Redemption | `redemption_count` | 쿠폰 Redemption 횟수 | count(coupon redemptions) |
| Outcome | Redemption | `redemption_any` | 쿠폰 Redemption 여부 (binary) | 1 if redemption_count > 0 else 0 |
| Outcome | Purchase | `purchase_amount` | 캠페인 기간 구매 금액 | sum(SALES_VALUE) in campaign period |
| Outcome | Purchase | `purchase_count` | 캠페인 기간 방문 횟수 | nunique(BASKET_ID) in campaign period |
| **Demographic** | Age | `AGE_DESC` | 연령대 | 19-24, 25-34, 35-44, 45-54, 55-64, 65+ |
| Demographic | Marital | `MARITAL_STATUS_CODE` | 결혼 상태 | A(Single), B(Married), U(Unknown) |
| Demographic | Income | `INCOME_DESC` | 소득 수준 | Under 15K ~ 250K+ |
| Demographic | Home | `HOMEOWNER_DESC` | 주택 소유 | Homeowner, Renter, Unknown |
| Demographic | Composition | `HH_COMP_DESC` | 가구 구성 | 1 Adult, 2 Adults, ... |
| Demographic | Size | `HOUSEHOLD_SIZE_DESC` | 가구 규모 | 1, 2, 3, 4, 5+ |
| Demographic | Kids | `KID_CATEGORY_DESC` | 자녀 유무 | None, 1, 2, 3+ |

---

#### Value × Need Feature 할당

| Layer | Category | Feature 수 | 변수 |
|-------|----------|-----------|------|
| **Value** | Recency | 6 | recency, recency_weeks, active_4w/12w, days_between_* |
| Value | Frequency | 6 | frequency, frequency_per_*, transaction_count, weeks_with_purchase, purchase_regularity |
| Value | Monetary | 7 | monetary_*, coupon_savings_ratio |
| Value | Time | 1 | week_coverage |
| | **Value 합계** | **20** | |
| **Need** | Behavioral | 7 | discount_*, private_label_ratio, n_*, avg_*_per_basket |
| Need | Category | 6 | share_* |
| | **Need 합계** | **13** | |

### 3.4 Evaluation

**Technical Metrics:**
- NMF: Reconstruction Error, Sparsity
- Clustering: Silhouette Score, Calinski-Harabasz Index, Davies-Bouldin Index

**Interpretability:**
- Factor Loading 해석 가능성
- Cluster 분리도 (ANOVA, Chi-square Test)

**Business:**
- 세그먼트 간 CLV 차이 유의성
- 세그먼트 정의의 실행 가능성

**Stability:**
- 기간 간 세그먼트 일관성 (Adjusted Rand Index)
- 고객 세그먼트 이동률

---

## 4. Track 2: Causal Targeting

### 4.1 Methods

#### Step 2.1: HTE Analysis

**목표:** 캠페인의 Heterogeneous Treatment Effect 추정.

**Methods:**
- Meta-learner: S-Learner, T-Learner, X-Learner, R-Learner
- Tree 기반: Causal Forest (GRF) - 규칙 추출 가능

**Treatment 정의:**
- Campaign Targeting (TypeA: 타겟팅, TypeB/C: 균일)
- Coupon 발행

**Outcome 변수:**
- 쿠폰 Redemption
- 구매 금액 증가
- Category 구매

**Covariates:**
- Track 1 Feature (RFM, Behavioral, Category)
- Track 1 Segment (Moderator로 활용)
- Demographics (hh_demographic)
- 마케팅 노출 (display, mailer)

**분석 질문:**
1. Treatment Effect Heterogeneity가 존재하는가? (BLP Test)
2. 어떤 고객 특성이 효과 차이를 유발하는가?
3. CATE 분포는 어떠한가?

#### Step 2.2: Optimal Policy

**목표:** 최적 타겟팅 규칙 도출 및 Policy Value 추정.

**Methods:**
- Subgroup Discovery: Causal Tree 규칙
- Policy Learning: CATE 기반 최적 Treatment 할당
- Policy Value Estimation: Policy 하의 기대 Outcome

**Output:**
- Targeting Rules (해석 가능한 의사결정 규칙)
- Policy Value (랜덤 타겟팅 대비 기대 개선)
- 최적 Policy 하의 ROI 추정

### 4.2 Operational Definitions (2 Scenarios)

**Campaign 데이터 개요:**

| 유형 | 캠페인 수 | 타겟팅 레코드 수 | 캠페인당 평균 |
|------|----------|-----------------|--------------|
| TypeA | 5 | 3,979 | ~796 |
| TypeB | 19 | 2,655 | ~140 |
| TypeC | 6 | 574 | ~96 |

---

#### Scenario 1: TypeA Only (Primary) - **First Campaign Only**

**설계 근거:** Clean Causal Identification을 위해 각 고객은 **첫 번째 TypeA 캠페인**만 사용하여 정확히 한 번 등장한다. 이는 이전 캠페인으로 인한 Pre-treatment Contamination을 방지한다.

| 항목 | 정의 |
|------|------|
| **Analysis Unit** | `household_key` (고객) |
| **Treatment** | 고객의 **첫** TypeA 캠페인 타겟팅 (1,513명) |
| **Control** | 어떤 TypeA 캠페인에도 타겟팅되지 않음 (987명) |
| **Total Sample** | 2,500 관측치 (고객당 1회) |

**First Campaign 분포:**

| Campaign | Week | Treatment 수 |
|----------|------|-------------|
| 26 | 32-38 | 332 |
| 30 | 47-53 | 126 |
| 8 | 59-66 | 768 |
| 13 | 72-79 | 153 |
| 18 | 84-92 | 134 |

**기간 설계 (캠페인별):**

| Campaign | Pre-treatment | Outcome Period |
|----------|---------------|----------------|
| 26 | Week 1-31 | Week 32-42 |
| 30 | Week 1-46 | Week 47-53 |
| 8 | Week 1-58 | Week 59-70 |
| 13 | Week 1-71 | Week 72-83 |
| 18 | Week 1-83 | Week 84-96 |

**왜 First Campaign Only인가?**
- Pre-treatment Contamination 방지: Campaign 30의 Pre-treatment (Week 1-46)는 모든 캠페인을 사용하면 Campaign 26 기간 (Week 32-38)을 포함
- 각 고객이 정확히 한 번 등장 → 독립 관측
- Carry-over Effect 없이 깨끗한 Identification
- 62% 샘플 감소 (3,979 → 1,513 Treatment)이지만 더 깨끗한 추정치

---

#### Scenario 2: First Campaign Overall with Type as Attribute

**설계 근거:** 캠페인 **유형 비교**와 함께 Clean Causal Identification을 위해 각 고객은 **첫 번째 캠페인** (모든 유형)만 사용하여 정확히 한 번 등장한다. `campaign_type`은 HTE 분석을 위한 속성으로 기록된다.

| 항목 | 정의 |
|------|------|
| **Analysis Unit** | `household_key` (고객) |
| **Treatment** | 고객의 **첫** 캠페인 타겟팅 (모든 유형) |
| **Control** | 어떤 캠페인에도 타겟팅되지 않음 |
| **Moderator** | `campaign_type` (TypeA/TypeB/TypeC as attribute) |
| **Total Sample** | ~2,484 관측치 (고객당 1회) |

**First Campaign Type 분포 (추정):**

| First Campaign Type | Treatment 수 |
|--------------------|-------------|
| TypeA | ~500-600 |
| TypeB | ~700-800 |
| TypeC | ~200-300 |
| Control (타겟팅 없음) | ~900 |

**기간 설계 (캠페인별):**
- Pre-treatment: Week 1 ~ (첫 캠페인 시작 - 1)
- Outcome: 첫 캠페인 시작 ~ 종료 + 4주 (다음 캠페인으로 제한)

**연구 질문:**
1. 캠페인 유형 간 Treatment Effect가 다른가?
2. 어떤 캠페인 유형이 어떤 고객 세그먼트에 가장 효과적인가?
3. 고객 특성과 캠페인 유형 간 Interaction이 있는가?

**왜 First Campaign Overall인가?**
- 모든 캠페인 유형이 시간적으로 겹칠 수 있음 (TypeA, TypeB, TypeC 동시 실행)
- "유형별 첫 캠페인" 사용 시 여전히 Contamination 발생 (예: TypeA 이후 TypeB)
- 고객당 단일 관측 + campaign_type as attribute로 깨끗한 유형 비교 가능

---

#### Common Definitions (전체 Scenario)

| 항목 | 정의 |
|------|------|
| **Outcome** | `redemption` (쿠폰 사용), `purchase_amount` (Outcome 기간) |
| **Confounders** | Pre-treatment Track 1 Feature, 마케팅 노출 경향, Demographics |

#### Scenario 선택 가이드

| 연구 질문 | 권장 Scenario |
|----------|--------------|
| "TypeA 캠페인의 Causal Effect는 무엇인가?" | **Scenario 1 (First TypeA Only)** |
| "캠페인 유형 간 효과는 어떻게 다른가?" | **Scenario 2 (First Campaign Overall)** |
| "어떤 고객이 TypeA에 반응하는가?" (HTE) | Scenario 1 |
| "어떤 캠페인 유형이 가장 효과적인가?" (Type HTE) | Scenario 2 |

### 4.3 Causal Graph

#### DAG 구조

```
                    Marketing Exposure (display, mailer)
                           │
                           ↓
Campaign Targeting ──────→ Purchase ←────── Customer Characteristics
         │                                     (confounders)
         │                                          │
         └──────→ Coupon Redemption ←───────────────┘
```

#### 변수 역할

| 역할 | 변수 |
|------|------|
| **Treatment** | Campaign Targeting (TypeA), Coupon 발행 |
| **Confounders** | 마케팅 노출 (display, mailer), 고객 특성 (Demographics, 과거 행동) |
| **Outcome** | 쿠폰 Redemption, 구매 금액, Category 구매 |
| **Effect Modifiers** | 모든 Covariate (HTE 분석용) |

#### Identification 전략

**가정:**
1. **Unconfoundedness**: 관측된 Covariate 조건 하에서 Treatment 할당이 Potential Outcome과 독립
2. **Overlap**: 모든 고객이 Treatment를 받을 양의 확률 보유

**Debiasing 접근법:**
- 마케팅 노출 (display, mailer)을 Confounder로 포함
- Pre-treatment 고객 특성 통제
- 내생성 회피를 위해 Pre-campaign 기간 (Week 1-31) Feature 사용

### 4.4 Features

**Treatment 변수:**
- `campaign_targeted`: 캠페인 타겟팅 지표 (binary)
- `campaign_type`: 캠페인 유형 (A/B/C)

**Confounders - 마케팅 노출 (causal_data에서, Pre-treatment Period Week 1-31):**
- `display_exposure_rate`: Display 노출 구매 비율
- `mailer_exposure_rate`: Mailer 노출 구매 비율
- `mailer_type_A/B/C`: 각 Mailer 유형 노출
- **참고**: 이 Feature들은 Treatment 자체가 아닌 고객의 Display/Mailer 노출 상품 구매 *경향*을 측정. 내생성 회피를 위해 Pre-treatment 기간에서 측정.

**Confounders - Demographics (hh_demographic에서):**
- AGE_GRP, INCOME_GRP, HOUSEHOLD_SIZE
- KID_CATEGORY_DESC, MARITAL_STATUS
- CLASSIFICATION_1~7

**Outcome 변수 (Campaign Period Week 32-102):**
- `redemption`: 쿠폰 Redemption 지표
- `purchase_amount`: 캠페인 기간 구매 금액
- `category_purchase`: Target Category 구매 지표

### 4.5 Evaluation

**Causal Metrics:**
- ATE 유의성 (Bootstrap CI, p-value)
- HTE 존재 테스트 (BLP Test, Calibration)
- CATE 분포 분석

**Prediction Metrics:**
- Uplift Curve (AUUC, Qini Coefficient)
- Calibration Plot

**Policy Metrics:**
- Policy Value (Policy 하의 기대 Outcome)
- Targeting Efficiency (precision@k)
- 최적 Policy vs 랜덤 타겟팅 대비 ROI

**Robustness & Validation:**
- Bootstrap Variance
- Cross-validation Stability
- 모델 비교 (다른 모델 간 CATE Correlation)

**Causal Validation (Ground Truth 없음):**
- **Refutation Tests:**
  - Placebo Treatment: 랜덤 Treatment → Effect ≈ 0
  - Random Confounder: 랜덤 변수 추가 → Effect 불변
  - Subset Validation: 데이터 부분집합에서 유사한 결과
- **Sensitivity Analysis:**
  - E-value: 얼마나 강한 Unmeasured Confounding이 필요한가?
  - Rosenbaum Bounds: Hidden Bias에 대한 민감도
- **Calibration:**
  - RATE (Sorted Group ATE): 높은 CATE 그룹이 실제로 더 높은 효과를 보이는가?

---

## 5. 기간 및 검증 설계

### 5.1 데이터 가용성

| 데이터 | 기간 | 비고 |
|--------|------|------|
| Transactions | Day 1-711 (Week 1-102) | 전체 2년 |
| Campaigns | Day 224-719 (Week 32-102) | 후반 기간에만 캠페인 |
| causal_data | Week 9-101 | 마케팅 노출 데이터 |

**주요 기간:**
- Week 1-31: Pre-campaign 기간 (캠페인 없음)
- Week 32-102: Campaign 기간

### 5.2 Track 1 기간 설계

| 설정 | 값 | 근거 |
|------|-----|------|
| **Analysis Period** | Week 1-102 (전체) | 최대 데이터, 완전한 고객 행동 |
| **Stability Check** | 반기 비교 | 시간에 따른 세그먼트 일관성 검증 |

### 5.3 Track 2 기간 설계

| 설정 | 값 | 근거 |
|------|-----|------|
| **Analysis Period** | Week 32-102 | Campaign 기간만 (Treatment 데이터 가용) |
| **Pre-treatment Features** | Week 1-31 | Pre-campaign 행동을 Confounder로 사용 (정보 누출 방지) |
| **Validation** | TBD | 옵션: 시간 기반 분할, 캠페인 기반 분할, Cross-validation |

### 5.4 고객 커버리지

| 기간 | Weeks | 고객 수 | 커버리지 |
|------|-------|--------|---------|
| 전체 (1-102) | 102 | 2,500 | 100% |
| Campaign (32-102) | 71 | 2,490 | 99.6% |
| 최근 52주 (51-102) | 52 | 2,479 | 99.2% |

---

## 6. 구현 계획

### 6.1 Notebooks

| Notebook | Track | 내용 |
|----------|-------|------|
| `00_study_design.ipynb` | - | Study Design 문서화 (본 노트북) |
| `01_feature_engineering.ipynb` | 1 & 2 | 전체 고객 Feature 구축 |
| `02_customer_profiling.ipynb` | 1.1 | NMF + Clustering |
| `03_clv_behavior_segments.ipynb` | 1.2 | CLV × Behavior Segmentation |
| `04_hte.ipynb` | 2.1 | HTE 분석 |
| `05_optimal_policy.ipynb` | 2.2 | Optimal Policy Learning |

### 6.2 구현 순서

```
Track 1 (우선순위: 높음)
────────────────────────
1. Feature Engineering (01_feature_engineering.ipynb)
   - 33개 전체 Base Feature 구축
   - Value 및 Need Feature 그룹 구현

2. Step 1.1 Customer Profiling (02_customer_profiling.ipynb)
   - NMF Factor 분석
   - Base Clustering

3. Step 1.2 CLV × Behavior (03_clv_behavior_segments.ipynb)
   - CLV 비교
   - Value-Need Integration

Track 2 (우선순위: 중간)
────────────────────────
4. Step 2.1 HTE Analysis (04_hte.ipynb)
   - Campaign Feature 구축
   - CATE 추정
   - Heterogeneity Testing

5. Step 2.2 Optimal Policy (05_optimal_policy.ipynb)
   - Subgroup Discovery
   - Policy Evaluation
```

---

## 7. Cohort 정의 및 비교

이 섹션에서는 데이터를 로드하고, `src/cohorts.py`를 사용하여 Cohort를 구축하고, 비교한다.

In [1]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd

from projects.segmentation_causal_targeting_dunnhumby.src.cohorts import (
    define_track1_cohort,
    define_track2_base_cohort,
    build_scenario1_cohort,
    build_scenario2_cohort,
    compare_cohorts
)

/opt/anaconda3/envs/side-projects/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-13 09:49:19,697	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# Load data
DATA_PATH = Path('.').absolute().parents[2] / 'data' / 'dunnhumby' / 'raw'

df_trans = pd.read_csv(DATA_PATH / 'transaction_data.csv')
df_campaign_table = pd.read_csv(DATA_PATH / 'campaign_table.csv')
df_campaign_desc = pd.read_csv(DATA_PATH / 'campaign_desc.csv')

print(f"Transactions: {len(df_trans):,} rows")
print(f"Campaign targeting: {len(df_campaign_table):,} rows")
print(f"Campaigns: {len(df_campaign_desc)} campaigns")

Transactions: 2,595,732 rows
Campaign targeting: 7,208 rows
Campaigns: 30 campaigns


In [3]:
# Build Track 1 Cohort
track1_cohort = define_track1_cohort(df_trans)
print(f"Track 1 Cohort: {len(track1_cohort):,} customers")

Track 1 Cohort: 2,500 customers


In [4]:
# Build Track 2 Base Cohort
track2_base = define_track2_base_cohort(df_trans)
print(f"Track 2 Base Cohort: {len(track2_base):,} customers")
print(f"  (requires purchase in Week 1-31 AND Week 32-102)")

Track 2 Base Cohort: 2,484 customers
  (requires purchase in Week 1-31 AND Week 32-102)


In [5]:
# Build Scenario 1: TypeA Only
scenario1 = build_scenario1_cohort(df_trans, df_campaign_table, df_campaign_desc)
print(f"Scenario 1 (TypeA Only):")
print(f"  Total rows: {len(scenario1):,}")
print(f"  Customers: {scenario1['household_key'].nunique():,}")
print(f"  Treatment: {scenario1['targeted'].sum():,} ({scenario1['targeted'].mean()*100:.1f}%)")
print(f"  Control: {(1-scenario1['targeted']).sum():,}")

Scenario 1 (TypeA Only):
  Total rows: 12,420
  Customers: 2,484
  Treatment: 3,978 (32.0%)
  Control: 8,442


In [6]:
# Build Scenario 2: All Types with Moderator
scenario2 = build_scenario2_cohort(df_trans, df_campaign_table, df_campaign_desc)
print(f"Scenario 2 (All Types):")
print(f"  Total rows: {len(scenario2):,}")
print(f"  Customers: {scenario2['household_key'].nunique():,}")
print(f"  Treatment: {scenario2['targeted'].sum():,} ({scenario2['targeted'].mean()*100:.1f}%)")
print(f"\n  By campaign type:")
print(scenario2.groupby('campaign_type')['targeted'].agg(['sum', 'mean']).round(3))

Scenario 2 (All Types):
  Total rows: 74,520
  Customers: 2,484
  Treatment: 7,206 (9.7%)

  By campaign type:
                sum   mean
campaign_type             
TypeA          3978  0.320
TypeB          2654  0.056
TypeC           574  0.039


In [7]:
# Cohort Comparison Summary
print("=" * 60)
print("COHORT COMPARISON SUMMARY")
print("=" * 60)

comparison = pd.DataFrame([
    {
        'Cohort': 'Track 1',
        'Unit': 'Customer',
        'N_Rows': len(track1_cohort),
        'N_Customers': len(track1_cohort),
        'N_Treatment': '-',
        'Treatment_%': '-'
    },
    {
        'Cohort': 'Track 2 Base',
        'Unit': 'Customer',
        'N_Rows': len(track2_base),
        'N_Customers': len(track2_base),
        'N_Treatment': '-',
        'Treatment_%': '-'
    },
    {
        'Cohort': 'Scenario 1 (TypeA)',
        'Unit': 'Customer × Campaign',
        'N_Rows': len(scenario1),
        'N_Customers': scenario1['household_key'].nunique(),
        'N_Treatment': scenario1['targeted'].sum(),
        'Treatment_%': f"{scenario1['targeted'].mean()*100:.1f}%"
    },
    {
        'Cohort': 'Scenario 2 (All)',
        'Unit': 'Customer × Campaign',
        'N_Rows': len(scenario2),
        'N_Customers': scenario2['household_key'].nunique(),
        'N_Treatment': scenario2['targeted'].sum(),
        'Treatment_%': f"{scenario2['targeted'].mean()*100:.1f}%"
    }
])

comparison

COHORT COMPARISON SUMMARY


,Cohort,Unit,N_Rows,N_Customers,N_Treatment,Treatment_%
0,Track 1,Customer,2500,2500,-,-
1,Track 2 Base,Customer,2484,2484,-,-
2,Scenario 1 (TypeA),Customer × Campaign,12420,2484,3978,32.0%
3,Scenario 2 (All),Customer × Campaign,74520,2484,7206,9.7%
